In [ ]:
# @title Preparação de Dados (Autoencoder)

import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import os


# --- Definição do Caminho ---
DATA_PATH = '/data/mocks'


# Arquivos de entrada do Integrante 1 (Contrato de Interface)
X_TRAIN_FILE = os.path.join(DATA_PATH, 'X_train_processed.csv')
Y_TRAIN_FILE = os.path.join(DATA_PATH, 'Y_train.csv')


# --- 1. Carregamento dos Dados ---
print("1. Carregando dados de treino (Features e Rótulos)...")
X_train_full = pd.read_csv(X_TRAIN_FILE)
Y_train_full = pd.read_csv(Y_TRAIN_FILE)


# Garantir que o rótulo seja uma série para fácil indexação
y_train_series = Y_train_full['Class']


# Relatório inicial
print(f"Dimensão Total do Conjunto de Treino Recebido: {X_train_full.shape}")
print(f"Distribuição de Classes no Treino (Completo):\n{y_train_series.value_counts()}")


# --- 2. Filtro de Classe (Classe 0) ---
print("\n2. Filtrando o conjunto para manter APENAS a Classe 0 (Normal)...")

# Cria uma máscara booleana para selecionar apenas as observações normais (Classe 0)
mask_normal = (y_train_series == 0)

# Aplica a máscara para obter apenas os dados normais (Normais = Classe 0)
X_normal = X_train_full[mask_normal]
y_normal = y_train_series[mask_normal] # Rótulos dos dados normais (todos são 0)

X_anomaly_check = X_train_full[~mask_normal] # Isso é apenas a classe 1 (fraude)

print(f"Total de Transações Normais (Classe 0): {X_normal.shape[0]}")
print(f"Total de Anomalias (Classe 1): {X_anomaly_check.shape[0]}")


# --- 3. Divisão Interna: Treino Puro vs. Validação Interna ---

# Usaremos uma proporção de 70% para Treino Puro e 30% dos Normais restantes
# para o Conjunto de Validação. Adicionaremos as anomalias neste conjunto.

TEST_SIZE = 0.3  # 30% para o conjunto de Validação Interna
RANDOM_STATE = 42 # Para garantir reprodutibilidade

# Divide a base normal (Classe 0) em duas partes
X_train_pure, X_val_normal, y_train_pure, y_val_normal = train_test_split(
    X_normal,
    y_normal,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE
)

# --- 4. Montagem do Conjunto de Validação Interna ---
print("\n4. Montando o Conjunto de Validação Interna (Contém 0s e 1s)...")

# O conjunto de validação interna deve conter amostras Normais E Anômalas
# para que possamos monitorar o desempenho de classificação/separação durante a tunagem.
X_val_combined = pd.concat([X_val_normal, X_anomaly_check], ignore_index=True)
y_val_combined = pd.concat([y_val_normal, y_train_series[~mask_normal]], ignore_index=True)

# Relatório Final
print("\n--- RESUMO FINAL DOS CONJUNTOS INTERNOS ---")
print(f"1. Conjunto de Treino PURO (AE.fit()): {X_train_pure.shape}")
print(f"   -> Contém APENAS Classe: {y_train_pure.unique()}")
print(f"2. Conjunto de Validação INTERNA (Monitoramento): {X_val_combined.shape}")
print(f"   -> Contém Classe 0 e Classe 1: {y_val_combined.value_counts().to_dict()}")

# Os objetos prontos para a próxima fase são: X_train_pure (para fit) e X_val_combined/y_val_combined (para tunagem)
# X_train_pure é o que você usará no model.fit(X_train_pure, X_train_pure, ...)

In [ ]:
# @title Treinamento e Otimização da Arquitetura

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import average_precision_score, roc_auc_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# --- Configurações Iniciais ---
# Se estiver no Colab/Jupyter, certifique-se de que os objetos da Fase 1 (X_train_pure, X_val_combined, y_val_combined)
# estão na memória. Se não, carregue-os ou rode a célula da Fase 1 novamente.

# Parâmetros Fixos
INPUT_DIM = X_train_pure.shape[1] # Número de features (30 no mock)
RANDOM_SEED = 42
tf.random.set_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# --- Definição do Espaço de Busca (Grid Search) ---
# Vamos testar diferentes tamanhos de gargalo e taxas de aprendizado
param_grid = {
    'encoding_dim': [4, 8, 16], # Tamanho do Gargalo (Bottleneck)
    'learning_rate': [0.01, 0.001],
    'epochs': [50], # Fixo, controlado pelo Early Stopping
    'batch_size': [32, 64]
}

# --- Função para Construir o Autoencoder ---
def build_autoencoder(input_dim, encoding_dim):
    """
    Cria um Autoencoder simples: Input -> Encoder -> Bottleneck -> Decoder -> Output
    """
    input_layer = Input(shape=(input_dim,))

    # Encoder (Compressão)
    # Adicionamos uma camada intermediária para dar profundidade (opcional, mas bom)
    encoded = Dense(int(input_dim * 0.75), activation='relu')(input_layer)

    # Bottleneck (Gargalo)
    bottleneck = Dense(encoding_dim, activation='relu')(encoded)

    # Decoder (Reconstrução)
    decoded = Dense(int(input_dim * 0.75), activation='relu')(bottleneck)
    output_layer = Dense(input_dim, activation='linear')(decoded) # Linear ou Sigmoid dependendo da normalização (0-1 -> Sigmoid)
    # Nota: Se seus dados forem Padronizados (StandardScaler, média 0), use 'linear'.
    # Se forem Normalizados (MinMax, 0 a 1), use 'sigmoid'. Vamos assumir 'sigmoid' para o mock (0-1).

    autoencoder = Model(inputs=input_layer, outputs=output_layer)
    return autoencoder

# --- Loop de Treinamento e Validação (Grid Search Manual) ---
results = []

print(f"Iniciando Grid Search com {len(param_grid['encoding_dim']) * len(param_grid['learning_rate']) * len(param_grid['batch_size'])} combinações...")

for encoding_dim in param_grid['encoding_dim']:
    for lr in param_grid['learning_rate']:
        for batch in param_grid['batch_size']:

            print(f"\n--- Testando: Gargalo={encoding_dim}, LR={lr}, Batch={batch} ---")

            # 1. Construir Modelo
            autoencoder = build_autoencoder(INPUT_DIM, encoding_dim)

            # 2. Compilar (Otimizador Adam + MSE Loss)
            optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
            autoencoder.compile(optimizer=optimizer, loss='mean_squared_error')

            # 3. Early Stopping (Monitorar Validação Interna)
            # Nota: No fit, usaremos X_val_combined como validation_data.
            # O Keras calculará o loss (reconstrução) nele.
            # Como X_val_combined tem anomalias, o loss de validação será naturalmente mais alto que o treino,
            # mas o Early Stopping ainda funciona para detectar quando a rede para de aprender o padrão geral.
            early_stop = EarlyStopping(
                monitor='val_loss',
                patience=5,
                mode='min',
                restore_best_weights=True,
                verbose=0
            )

            # 4. Treinar (Apenas com X_train_pure - Classe 0)
            history = autoencoder.fit(
                X_train_pure, X_train_pure, # Entrada = Saída (Reconstrução)
                epochs=param_grid['epochs'][0],
                batch_size=batch,
                shuffle=True,
                validation_data=(X_val_combined, X_val_combined),
                callbacks=[early_stop],
                verbose=0 # Silencioso para não poluir o output
            )

            # 5. Avaliar Performance no Conjunto de Validação Interna
            # Gerar reconstruções
            reconstructions = autoencoder.predict(X_val_combined, verbose=0)

            # Calcular Erro de Reconstrução (MSE) por amostra
            # loss = mean((X - X_hat)^2, axis=1)
            mse = np.mean(np.power(X_val_combined - reconstructions, 2), axis=1)

            # Calcular Métricas de Detecção (AUC-PR é a mais importante para desbalanceado)
            # y_val_combined são os rótulos verdadeiros (0 ou 1)
            auc_pr = average_precision_score(y_val_combined, mse)
            auc_roc = roc_auc_score(y_val_combined, mse)

            print(f"   -> Val Loss Final: {history.history['val_loss'][-1]:.4f}")
            print(f"   -> AUC-PR (Validação): {auc_pr:.4f}")

            # Guardar resultados
            results.append({
                'encoding_dim': encoding_dim,
                'learning_rate': lr,
                'batch_size': batch,
                'auc_pr': auc_pr,
                'auc_roc': auc_roc,
                'model': autoencoder # Guardar o objeto do modelo (cuidado com memória em loops grandes)
            })

# --- Seleção do Melhor Modelo ---
# Ordenar por AUC-PR decrescente
results_df = pd.DataFrame(results)
best_run = results_df.sort_values(by='auc_pr', ascending=False).iloc[0]

print("\n=============================================")
print("🏆 MELHOR MODELO ENCONTRADO")
print("=============================================")
print(f"Gargalo (Bottleneck): {best_run['encoding_dim']}")
print(f"Learning Rate: {best_run['learning_rate']}")
print(f"Batch Size: {best_run['batch_size']}")
print(f"AUC-PR (Validação): {best_run['auc_pr']:.4f}")
print(f"AUC-ROC (Validação): {best_run['auc_roc']:.4f}")

# Recuperar o melhor modelo treinado
best_autoencoder = best_run['model']

# --- Salvar o Modelo (Opcional) ---
# best_autoencoder.save('best_autoencoder_model.h5')

# --- Visualização do Erro de Reconstrução (Melhor Modelo) ---
# Vamos plotar a distribuição dos erros para Normais vs Anomalias no conjunto de validação
reconstructions_val = best_autoencoder.predict(X_val_combined, verbose=0)
mse_val = np.mean(np.power(X_val_combined - reconstructions_val, 2), axis=1)

error_df = pd.DataFrame({'reconstruction_error': mse_val, 'true_class': y_val_combined})

plt.figure(figsize=(10, 6))
plt.title("Distribuição do Erro de Reconstrução (Validação Interna)")
for label in [0, 1]:
    subset = error_df[error_df['true_class'] == label]
    plt.hist(subset['reconstruction_error'], bins=50, alpha=0.6, label=f"Classe {label}", density=True)

plt.xlabel("Erro de Reconstrução (MSE)")
plt.ylabel("Densidade")
plt.legend()
plt.show()

In [ ]:
# @title Geração do Score Final (Conjunto de Teste)

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import precision_recall_curve, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import os

# --- Configurações Iniciais ---
# Se estiver no Colab/Jupyter, certifique-se de que o 'best_autoencoder' da Fase 2 está na memória.
# Se não, carregue o modelo salvo: best_autoencoder = tf.keras.models.load_model('best_autoencoder_model.h5')

# Parâmetros
DATA_PATH = 'data/mocks' # ou 'data/processed'
OUTPUT_PATH = 'outputs'

if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

# Arquivos de Teste (Contrato de Interface)
X_TEST_FILE = os.path.join(DATA_PATH, 'X_test_processed.csv')
Y_TEST_FILE = os.path.join(DATA_PATH, 'Y_test.csv')
IDS_TEST_FILE = os.path.join(DATA_PATH, 'ids_test.csv')

# --- 1. Carregamento dos Dados de Teste ---
print("1. Carregando dados de Teste...")
X_test = pd.read_csv(X_TEST_FILE)
y_test = pd.read_csv(Y_TEST_FILE)['Class'] # Gabarito (apenas para definir threshold ótimo, não para treino!)
ids_test = pd.read_csv(IDS_TEST_FILE)['id']

print(f"Dimensão do Teste: {X_test.shape}")

# --- 2. Geração do Anomaly Score (Erro de Reconstrução) ---
print("\n2. Calculando Anomaly Score (Erro de Reconstrução)...")

# Passar o X_test pelo melhor modelo
reconstructions_test = best_autoencoder.predict(X_test, verbose=0)

# Calcular MSE (Mean Squared Error) por amostra
# Score = média do erro quadrado de todas as features daquela transação
anomaly_scores = np.mean(np.power(X_test - reconstructions_test, 2), axis=1)

print(f"Scores calculados. Min: {anomaly_scores.min():.4f}, Max: {anomaly_scores.max():.4f}")

# --- 3. Definição do Threshold Ótimo (Usando Precision-Recall) ---
print("\n3. Definindo Threshold Ótimo...")

# Como os dados são desbalanceados, a Curva Precision-Recall é melhor que a ROC para achar o corte.
precision, recall, thresholds = precision_recall_curve(y_test, anomaly_scores)

# Estratégia: Encontrar o threshold que maximiza o F1-Score
# F1 = 2 * (Precision * Recall) / (Precision + Recall)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-10) # 1e-10 evita divisão por zero
best_idx = np.argmax(f1_scores)
best_threshold = thresholds[best_idx]
best_f1 = f1_scores[best_idx]

print(f"Threshold Ótimo encontrado: {best_threshold:.4f}")
print(f"F1-Score Estimado no Ponto Ótimo: {best_f1:.4f}")

# Plotar Curva Precision-Recall com o ponto ótimo
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, marker='.', label='Autoencoder')
plt.scatter(recall[best_idx], precision[best_idx], marker='o', color='red', label='Best Threshold', zorder=10)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Curva Precision-Recall (Seleção de Threshold)')
plt.legend()
plt.show()

# --- 4. Geração da Classificação Final (Is Anomaly?) ---
print("\n4. Gerando Classificação Final...")

# Aplica o threshold: Se score > threshold, é anomalia (1)
predictions = (anomaly_scores > best_threshold).astype(int)

# Relatório de Classificação
print("\n--- Relatório de Performance no Teste ---")
print(classification_report(y_test, predictions, target_names=['Normal', 'Fraude']))

# Matriz de Confusão
cm = confusion_matrix(y_test, predictions)
print(f"Matriz de Confusão:\n{cm}")

# --- 5. Exportação (Formato Contrato de Interface) ---
print("\n5. Exportando resultados para 'outputs/'...")

# Criar DataFrame final conforme contrato
df_output = pd.DataFrame({
    'id': ids_test,
    'anomaly_score': anomaly_scores, # Score contínuo (Crucial para AUC-ROC do Int. 1)
    'is_anomaly': predictions        # Predição binária baseada no seu threshold
})

# Salvar
output_file = os.path.join(OUTPUT_PATH, 'autoencoder_predictions.csv')
df_output.to_csv(output_file, index=False)

print(f"✅ Arquivo salvo com sucesso: {output_file}")
print("Exemplo das primeiras linhas:")
print(df_output.head())